In [57]:
import os
import pandas as pd
import glob
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
import concurrent.futures 
from pycaret.classification import *
from sklearn.metrics import accuracy_score
import requests
from bs4 import BeautifulSoup
import time
import random
from fuzzywuzzy import fuzz, process
from os import walk
from datetime import datetime

In [4]:
filenames = next(walk('C:/Users\idotz/OneDrive - Northeastern University/Python/Soccer Analysis'), (None, None, []))[2]
filenames = [f for f in filenames if '2022' in f and '_data' in f]

In [5]:
filenames

['20220826_data.pkl',
 '20220828_data.pkl',
 '20220829_data.pkl',
 '20220830_data.pkl',
 '20220831_data.pkl',
 '20220901_data.pkl',
 '20220902_data.pkl',
 '20220903_data.pkl',
 '20220904_data.pkl',
 '20220905_data.pkl',
 '20220906_data.pkl',
 '20220907_data.pkl',
 '20220908_data.pkl']

In [6]:
all_data = []
for f in filenames:
    open_file = open(f, "rb")
    d = pickle.load(open_file)
    all_data.append(d)
    open_file.close()
    
all_data = [i for j in all_data for i in j]

In [7]:
f = []
for dic in all_data:
    d = {}
    for k, v in dic.items():
        try:
            d[k] = float(v[0]) - float(v[1])
        except:
            d[k] = v
    f.append(d)


In [62]:
df = pd.DataFrame(f)
df = df[df['half_time'] == 'Half-Time']
df = df[(df['odds'] != '') & (df['odds'] != {})]
df = df.reset_index(drop = True)
df2 = pd.json_normalize(df['bet'])
df = pd.concat([df, df2], axis = 1)
df['o_names'] = df['odds_url'].apply(lambda x: ['draw'] + x.split('/')[-2].split('-v-'))
r = []
for i in df[['odds','o_names']].to_numpy():
    oo = [getname(name, list(i[0].keys())) for name in i[1]]
    oo = [i[0][key] for key in oo]
    r.append(oo)
df['oo'] = r
split_df = pd.DataFrame(df['oo'].tolist(), columns=['0', '1', '2'])
df = pd.concat([df, split_df], axis=1)
df['odds_predict'] = df['oo'].apply(lambda x: np.argmin(x))
df = df.drop(['match_name', 'half_time','url','odds_url','odds','o_names','oo','predicted_probs',
             'bet','draw','home','away','full_odds','winner'], axis = 1)

In [63]:
df['final_score'] = scores

In [15]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [14]:
def get_winner(url):
    page = requests.get(url)
    time.sleep(random.randint(0, 4))
    soup = BeautifulSoup(page.content, 'html.parser')
    score = soup.find_all("span", {"class": "css-bw7eig-topRow"})[0].text.split(' - ')
    score = int(score[0]) - int(score[1])
    score = winner(score)
    return score

In [20]:
scores = []
for i in range(0, len(df['url']), 4):
    x = list(df['url'][i:i + 4])
    print(x)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        r = executor.map(get_winner, x)
    for result in r:
        scores.append(result)

['https://www.fotmob.com/match/3888550/matchfacts/puebla-vs-fc-juarez', 'https://www.fotmob.com/match/3888551/matchfacts/mazatlan-fc-vs-cf-america', 'https://www.fotmob.com/match/3900373/matchfacts/fc-utrecht-vs-ajax', 'https://www.fotmob.com/match/3904422/matchfacts/nantes-vs-toulouse']
['https://www.fotmob.com/match/3902828/matchfacts/dundee-united-vs-celtic', 'https://www.fotmob.com/match/3916965/matchfacts/gent-vs-royal-antwerp', 'https://www.fotmob.com/match/3887414/matchfacts/randers-fc-vs-aalborg-bk', 'https://www.fotmob.com/match/3887416/matchfacts/viborg-vs-lyngby']
['https://www.fotmob.com/match/3905446/matchfacts/lugano-vs-st.-gallen', 'https://www.fotmob.com/match/3900374/matchfacts/excelsior-vs-psv-eindhoven', 'https://www.fotmob.com/match/3900375/matchfacts/fc-volendam-vs-fc-twente', 'https://www.fotmob.com/match/3900963/matchfacts/aston-villa-vs-west-ham-united']
['https://www.fotmob.com/match/3900971/matchfacts/wolverhampton-wanderers-vs-newcastle-united', 'https://www.

['https://www.fotmob.com/match/3888623/matchfacts/necaxa-vs-leon', 'https://www.fotmob.com/match/3900986/matchfacts/everton-vs-liverpool', 'https://www.fotmob.com/match/3902849/matchfacts/celtic-vs-rangers', 'https://www.fotmob.com/match/3917967/matchfacts/mallorca-vs-girona']
['https://www.fotmob.com/match/3919120/matchfacts/fiorentina-vs-juventus', 'https://www.fotmob.com/match/3903581/matchfacts/bayer-leverkusen-vs-sc-freiburg', 'https://www.fotmob.com/match/3903586/matchfacts/vfl-bochum-vs-werder-bremen', 'https://www.fotmob.com/match/3903582/matchfacts/union-berlin-vs-bayern-m%C3%BCnchen']
['https://www.fotmob.com/match/3903585/matchfacts/wolfsburg-vs-1.-fc-k%C3%B6ln', 'https://www.fotmob.com/match/3900983/matchfacts/brentford-vs-leeds-united', 'https://www.fotmob.com/match/3900985/matchfacts/chelsea-vs-west-ham-united', 'https://www.fotmob.com/match/3900988/matchfacts/newcastle-united-vs-crystal-palace']
['https://www.fotmob.com/match/3900990/matchfacts/tottenham-hotspur-vs-fulha

['https://www.fotmob.com/match/4010293/matchfacts/aek-larnaca-vs-rennes', 'https://www.fotmob.com/match/4010294/matchfacts/fenerbah%C3%A7e-vs-dynamo-kyiv', 'https://www.fotmob.com/match/4010532/matchfacts/hjk-vs-real-betis', 'https://www.fotmob.com/match/4010533/matchfacts/ludogorets-razgrad-vs-roma']
['https://www.fotmob.com/match/4010661/matchfacts/malm%C3%B6-ff-vs-braga', 'https://www.fotmob.com/match/4010662/matchfacts/union-berlin-vs-union-st.gilloise', 'https://www.fotmob.com/match/4010194/matchfacts/fiorentina-vs-rfs', 'https://www.fotmob.com/match/4010193/matchfacts/hearts-vs-istanbul-ba%C5%9Fak%C5%9Fehir']
['https://www.fotmob.com/match/4010217/matchfacts/anderlecht-vs-silkeborg', 'https://www.fotmob.com/match/4010243/matchfacts/austria-wien-vs-hapoel-beer-sheva', 'https://www.fotmob.com/match/4010242/matchfacts/villarreal-vs-lech-poznan', 'https://www.fotmob.com/match/4010268/matchfacts/slovacko-vs-partizan-beograd']
['https://www.fotmob.com/match/4010580/matchfacts/ballkani-

In [13]:
def winner(x):
    if x == 0:
        return 0
    elif x > 0:
        return 1
    else:
        return 2

In [64]:
accuracy_score(df['final_score'], df['odds_predict'])

0.6693877551020408

In [65]:
today = f"{datetime.now():%Y%m%d}"
print(today, df.shape)
df.to_csv(f'data_{today}.csv')

20220908 (245, 50)
